In [1]:
!pip install transformers accelerate bitsandbytes peft -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.0 MB/s eta 0:00:00


In [7]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [12]:
import json

with open("/content/drive/MyDrive/Deep_Model/myp_qa_data.json") as f:
    data = json.load(f)

for idx, entry in enumerate(data):
    user_msg = entry['messages'][0]['content'].lower()
    assistant_msg = entry['messages'][1]['content'].lower()

    # Check basic alignment
    if 'fund' in user_msg and 'fund' not in assistant_msg:
        print(f"Bad match at index {idx}:")
        print(f"Q: {user_msg}")
        print(f"A: {assistant_msg}\n")

Bad match at index 137:
Q: is using a home equity loan a smart way to fund renovations?
A: home equity loans can make sense if you have the equity and can manage the payments. ensure the renovations add enough value to justify the cost, and note that your home is the collateral if you default.

Bad match at index 156:
Q: how much should i keep in an emergency fund if i'm self-employed?
A: when self-employed, having 6–12 months of expenses saved may be wiser since income can be unpredictable. factor in business and personal obligations to decide the right cushion.

Bad match at index 171:
Q: what should i do with leftover 529 funds if my child gets a scholarship?
A: you can often change the beneficiary to another child or family member. some plans also allow penalty-free withdrawal of the scholarship amount (though you may owe taxes on earnings). verify your state’s rules and plan details.



In [13]:
# Check first 5 entries
print(json.dumps(data[:5], indent=2))

[
  {
    "messages": [
      {
        "role": "user",
        "content": "How should I start investing with $1000?"
      },
      {
        "role": "assistant",
        "content": "Begin by setting clear financial goals, then explore low-cost investment options like index funds or ETFs. Make sure you have an emergency fund in place before committing your entire sum to investments. Diversifying your portfolio can help mitigate risk."
      }
    ]
  },
  {
    "messages": [
      {
        "role": "user",
        "content": "What's a good strategy to pay off credit card debt quickly?"
      },
      {
        "role": "assistant",
        "content": "Pay more than the minimum whenever possible. Focus on high-interest balances first (the avalanche method), or tackle the smallest balances first for quick wins (the snowball method). Keeping credit utilization low and avoiding new debt is key."
      }
    ]
  },
  {
    "messages": [
      {
        "role": "user",
        "content": "Ho

In [14]:
!pip install -qU transformers accelerate peft bitsandbytes datasets
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, prepare_model_for_kbit_training
from datasets import Dataset
import json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 33.4 MB/s eta 0:00:00


In [15]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load tokenizer with proper chat template
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Add special tokens explicitly (critical for TinyLlama)
special_tokens = {
    "additional_special_tokens": ["<|system|>", "<|user|>", "<|assistant|>", "<|im_end|>"]
}
tokenizer.add_special_tokens(special_tokens)

# Verify tokens
print(f"User token ID: {tokenizer.convert_tokens_to_ids('<|user|>')}")  # Should be 32001
print(f"Asst token ID: {tokenizer.convert_tokens_to_ids('<|assistant|>')}")  # Should be 32002

User token ID: 32001
Asst token ID: 32002


In [17]:
def format_chat(example):
    # Convert to TinyLlama's required format
    formatted = []
    for msg in example["messages"]:
        formatted.append({
            "role": msg["role"],
            "content": msg["content"].strip() + "\n"  # Add newline for better tokenization
        })
    return {"messages": formatted}

def tokenize_function(examples):
    tokenized = tokenizer.apply_chat_template(
        examples["messages"],
        padding="max_length",
        truncation=True,
        max_length=512,  # Start smaller for stability
        return_tensors="pt",
        add_generation_prompt=False
    )

    # Create labels (mask user turns)
    labels = tokenized.clone()
    assistant_token = tokenizer.convert_tokens_to_ids("<|assistant|>")
    assistant_pos = (tokenized == assistant_token).nonzero()

    if len(assistant_pos) > 0:
        start_idx = assistant_pos[0,1].item() + 1  # Start after assistant token
        labels[:, :start_idx] = -100
    else:
        labels[:] = -100  # Invalid sample

    return {
        "input_ids": tokenized.squeeze(0),
        "labels": labels.squeeze(0)
    }

# Load and process data
with open("/content/drive/MyDrive/Deep_Model/myp_qa_data.json") as f:
    data = json.load(f)

dataset = Dataset.from_list(data)
dataset = dataset.map(format_chat)
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=False,  # Process one sample at a time
    remove_columns=dataset.column_names
)

# Split dataset
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [18]:
# Check first training sample
sample = tokenized_dataset["train"][0]
print("Decoded input:\n", tokenizer.decode(sample["input_ids"]))
print("\nLabels:")
print(sample["labels"])

# Should look like:
# Decoded input: <|user|> How to invest $1000?<|assistant|> Consider low-cost index funds...
# Labels: [-100, -100, ..., 2345, 6789, ...] (valid tokens only after assistant marker)

Decoded input:
 <|user|> 
Should I join a credit union if I'm eligible?
</s> 
<|assistant|> 
Credit unions often have strong community ties and lower fees. If you qualify and value personalized service, it can be a great choice. Make sure the credit union offers the services and convenience you need.
</s> 
</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>

In [19]:
# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
model.resize_token_embeddings(len(tokenizer))  # Critical for new tokens

# LoRA config
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()  # Should show ~0.1% params trainable

trainable params: 1,126,400 || all params: 1,101,191,168 || trainable%: 0.1023


In [20]:
training_args = TrainingArguments(
    output_dir="./financial-coach",
    num_train_epochs=5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    weight_decay=0.01,
    fp16=True,
    logging_steps=20,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    remove_unused_columns=False,
    report_to="none",
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False}
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)

# Critical final step
model.config.use_cache = False

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-20-195054e9c849>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.273800,0.212820
2,0.199900,0.198102
3,0.189000,0.195401
4,0.182400,0.193924


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  # support from version >= 0.6.2
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  # support from version >= 0.6.2
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  # support from version >= 0.6.2
/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: a3fc2251-d27d-449c-834e-0990f1a380dc)') - silently ignoring the lookup for the f

TrainOutput(global_step=110, training_loss=0.3852627949281172, metrics={'train_runtime': 151.2867, 'train_samples_per_second': 5.949, 'train_steps_per_second': 0.727, 'total_flos': 2748822488875008.0, 'train_loss': 0.3852627949281172, 'epoch': 4.8})

In [1]:
# Test
prompt = "What's the best strategy to handle leftover funds after paying bills?"
messages = [{"role": "user", "content": prompt}]

inputs = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    inputs,
    max_new_tokens=200,
    temperature=0.7,
    do_sample=True
)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

NameError: name 'tokenizer' is not defined